In [1]:
from raster_to_tile import data_split

tif_selva = r"D:\Shapefiles\PI\CO_2512022106159\VOL_PER1_ORT_001_010438\IMG_PER1_ORT_PMS_010438\IMG_PER1_20230922151914_ORT_PMS_010438.TIF"
tif_sierra = r"D:\Shapefiles\PI\CO_2510221339013\VOL_PER1_ORT_001_002705\IMG_PER1_ORT_PMS_002705\IMG_PER1_20210706153514_ORT_PMS_002705.TIF"

mask_casas_selva_path = r"D:\Diciembre\Entrenamiento\training_road_model\CapasSelva\buildings_32718_selva.gpkg"
mask_casas_sierra_path = r"D:\Diciembre\Entrenamiento\training_road_model\CapasSierra\buildings_32717_sierra.gpkg"

mask_caminos_selva_path = r"D:\Diciembre\Entrenamiento\training_road_model\CapasSelva\capa_carreteras_selva.gpkg"
mask_caminos_sierra_path = r"D:\Diciembre\Entrenamiento\training_road_model\CapasSierra\\capa_carreteras_sierra.gpkg"

gpkg_division_selva_tile = r"D:\Diciembre\Entrenamiento\training_road_model\CapasSelva\division_en_tile_selva.gpkg"
gpkg_division_sierra_tile = r"D:\Diciembre\Entrenamiento\training_road_model\CapasSierra\division_en_tile_sierra.gpkg"

dataset_selva_path = r"D:\Diciembre\Entrenamiento\training_road_model\DatasetSelvaMulticlase"
dataset_sierra_path = r"D:\Diciembre\Entrenamiento\training_road_model\DatasetSierraMulticlase1"

### Area Excluida
gpkg_area_ciudad = r"D:\Diciembre\Entrenamiento\training_road_model\CapasSierra\area_excluida_sierra.gpkg"

data_split(tif_sierra, mask_casas_sierra_path, mask_caminos_sierra_path, gpkg_division_sierra_tile, dataset_sierra_path, gpkg_area_excluida_path=gpkg_area_ciudad)

Leyendo raster entero para calcular percentiles para posterior normalización de parches
Cargando raster...
Cargando geometrías...
Geometria de : Casas
Geometria de : Caminos
Geometria de : Area fuera de interes
Cargando tiles...

Procesando tile 0 → train

Procesando tile 1 → test

Procesando tile 2 → train

Procesando tile 3 → val

Procesando tile 4 → train

Procesando tile 5 → val

Procesando tile 6 → train

Procesando tile 7 → train

Procesando tile 8 → train

Procesando tile 9 → train

Procesando tile 10 → train

Procesando tile 11 → test

Procesando tile 12 → train
Se excluyo el tile: 3064
Se excluyo el tile: 3081
Se excluyo el tile: 3081
Se excluyo el tile: 3098
Se excluyo el tile: 3098
Se excluyo el tile: 3115
Se excluyo el tile: 3115
Se excluyo el tile: 3132
Se excluyo el tile: 3132
Se excluyo el tile: 3149
Se excluyo el tile: 3149
Se excluyo el tile: 3166
Se excluyo el tile: 3166
Se excluyo el tile: 3183
Se excluyo el tile: 3183
Se excluyo el tile: 3200
Se excluyo el tile: 320

In [2]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
print("GPUs disponibles:", gpus)

GPUs disponibles: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [1]:
from utils2 import split_dataset
dataset_path = r"D:\Diciembre\Entrenamiento\training_road_model\DatasetSierraMulticlase"

(train_imgs, train_masks, val_imgs, val_masks, test_imgs, test_masks) = split_dataset(carpeta=dataset_path)


División correcta sin errores de archivos.
--------------------------------------------------
TRAIN     → imágenes: 4117 | máscaras: 4117
VALIDATION → imágenes: 938 | máscaras: 938
TEST      → imágenes: 969 | máscaras: 969
--------------------------------------------------
TOTAL IMGS: 6024
TOTAL MASKS: 6024
--------------------------------------------------


In [2]:
from TiffDataGeneratorAugmentedUnet import TIFFDataGeneratorAug

# Generadores
train_gen = TIFFDataGeneratorAug(train_imgs, train_masks, normalize='imagenet',batch_size=2, n_channels=3, augment=True)#<--- canales
val_gen = TIFFDataGeneratorAug(val_imgs, val_masks, normalize='imagenet', batch_size=2, shuffle=False, n_channels=3, augment=False)#<--- canales
test_gen = TIFFDataGeneratorAug(test_imgs, test_masks, normalize='imagenet', batch_size=1, shuffle=False, n_channels=3, augment=False)#<--- canales


c:\Users\prac-dnce\miniconda3\envs\tensorflow\lib\site-packages\albumentations\core\validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
d:\Diciembre\Entrenamiento\Unet\TiffDataGeneratorAugmentedUnet.py:49: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(3, 10), p=0.1)


In [4]:
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

from tensorflow import keras
import segmentation_models as sm

Segmentation Models: using `tf.keras` framework.


In [ ]:
def masked_sparse_cce(y_true, y_pred):
    """
    Sparse categorical crossentropy que ignora píxeles con label 255
    """
    y_true = tf.squeeze(y_true, axis=-1)  # (B,H,W)
    
    # Crear máscara para píxeles válidos
    mask = tf.not_equal(y_true, 255)
    mask = tf.cast(mask, tf.float32)
    
    # Reemplazar 255 con 0 para evitar errores
    y_true_safe = tf.where(mask > 0, y_true, 0)
    
    # Calcular loss por píxel
    loss = tf.keras.losses.sparse_categorical_crossentropy(
        y_true_safe, y_pred, from_logits=False
    )
    
    # Aplicar máscara y promediar solo sobre píxeles válidos
    loss = loss * mask
    
    # Evitar división por cero
    num_valid = tf.maximum(tf.reduce_sum(mask), 1.0)
    
    return tf.reduce_sum(loss) / num_valid

In [ ]:
class MaskedMeanIoU(tf.keras.metrics.Metric):
    def __init__(self, num_classes=3, name='masked_mean_iou', **kwargs):
        super().__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.total_cm = self.add_weight(
            name='total_confusion_matrix',
            shape=(num_classes, num_classes),
            initializer='zeros'
        )
    
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.squeeze(y_true, axis=-1)
        y_pred = tf.argmax(y_pred, axis=-1)
        
        # Convertir a int32 ANTES de crear la máscara
        y_true = tf.cast(y_true, tf.int32)
        y_pred = tf.cast(y_pred, tf.int32)
        
        # Máscara para ignorar 255
        mask = tf.not_equal(y_true, 255)
        
        # Aplicar máscara
        y_true = tf.boolean_mask(y_true, mask)
        y_pred = tf.boolean_mask(y_pred, mask)
        
        # Calcular confusion matrix
        current_cm = tf.math.confusion_matrix(
            y_true,
            y_pred,
            num_classes=self.num_classes,
            dtype=tf.float32
        )
        
        self.total_cm.assign_add(current_cm)
    
    def result(self):
        sum_over_row = tf.reduce_sum(self.total_cm, axis=0)
        sum_over_col = tf.reduce_sum(self.total_cm, axis=1)
        diag = tf.linalg.diag_part(self.total_cm)
        
        denominator = sum_over_row + sum_over_col - diag
        
        iou = tf.where(
            denominator > 0,
            diag / denominator,
            0.0
        )
        
        return tf.reduce_mean(iou)
    
    def reset_state(self):
        self.total_cm.assign(tf.zeros_like(self.total_cm))


In [ ]:
class MaskedSparseCategoricalAccuracy(tf.keras.metrics.Metric):
    def __init__(self, name='masked_acc', **kwargs):
        super().__init__(name=name, **kwargs)
        self.total_correct = self.add_weight(name='total_correct', initializer='zeros')
        self.total_pixels = self.add_weight(name='total_pixels', initializer='zeros')
    
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.squeeze(y_true, axis=-1)
        y_pred = tf.argmax(y_pred, axis=-1)
        
        # Convertir ambos a int32 antes de la máscara
        y_true = tf.cast(y_true, tf.int32)
        y_pred = tf.cast(y_pred, tf.int32)
        
        # Máscara para ignorar 255
        mask = tf.not_equal(y_true, 255)
        
        y_true = tf.boolean_mask(y_true, mask)
        y_pred = tf.boolean_mask(y_pred, mask)
        
        correct = tf.cast(tf.equal(y_true, y_pred), tf.float32)
        
        self.total_correct.assign_add(tf.reduce_sum(correct))
        self.total_pixels.assign_add(tf.cast(tf.size(y_true), tf.float32))
    
    def result(self):
        return self.total_correct / tf.maximum(self.total_pixels, 1.0)
    
    def reset_state(self):
        self.total_correct.assign(0.0)
        self.total_pixels.assign(0.0)

In [ ]:
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

BACKBONE = 'resnet34'#efficientnetb0 tmb puede ser (buena con imag satelitales)
model = sm.Unet(
    backbone_name=BACKBONE,
    encoder_weights='imagenet',
    classes=3,
    activation='softmax'
    #input_shape=(512, 512, 4)
)

model.compile(
    optimizer=Adam(1e-4),
    loss=masked_sparse_cce,
    metrics=[
        MaskedSparseCategoricalAccuracy(),
        MaskedMeanIoU(num_classes=3)
    ]
)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Early stopping: detener cuando el modelo deje de mejorar
early_stop = EarlyStopping(
    monitor='val_loss',      # métrica que se evalúa
    mode='min',           # porque cuanto más alto, mejor
    patience=10,
    restore_best_weights=True,
    verbose= 1
)

# Guardar el mejor modelo automáticamente
checkpoint = ModelCheckpoint(
    r"D:\Diciembre\Entrenamiento\training_road_model\Sierra\Checkpoints\checkpoint.keras",         # ruta del archivo
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=100,
    verbose=1,
    callbacks=[early_stop, checkpoint, reduce_lr]
)
print("Epochs realizadas:", len(history.history['loss']))

(2, 512, 512, 3)
Epoch 1/100
(2, 512, 512, 3)
(2, 512, 512, 3)


: 

In [ ]:
# Save final model
model.save("model_final.keras")

# Evaluate on train set
print("\n=== Evaluación en test set ===")
train_results = model.evaluate(train_gen, verbose=1)
print(f"Train Loss: {train_results[0]:.4f}")
print(f"Train Masked Accuracy: {train_results[1]:.4f}")
print(f"Train Masked Mean IoU: {train_results[2]:.4f}")
# Evaluate on test set
print("\n=== Evaluación en val set ===")
val_results = model.evaluate(val_gen, verbose=1)
print(f"Val Loss: {val_results[0]:.4f}")
print(f"Val Masked Accuracy: {val_results[1]:.4f}")
print(f"Val Masked Mean IoU: {val_results[2]:.4f}")
# Evaluate on test set
print("\n=== Evaluación en test set ===")
test_results = model.evaluate(test_gen, verbose=1)
print(f"Test Loss: {test_results[0]:.4f}")
print(f"Test Masked Accuracy: {test_results[1]:.4f}")
print(f"Test Masked Mean IoU: {test_results[2]:.4f}")